<a href="https://colab.research.google.com/github/BRAHIMLOUARDI/movie-web-app/blob/main/Welcome_To_Colaboratory1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q keras

In [10]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model,Sequential
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

# Path to translation filekk
path_to_data = 'fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs=pairs[0:192000]



In [ ]:
x=0
for index,pair in enumerate(pairs):
  print(pair[1])
  x=index
  print(x)

print(pairs[58270])


In [ ]:
!unzip  fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
def smart_find(haystack, needle):
    if haystack.startswith(needle+" "):
        return True
    if haystack.endswith(" "+needle):
        return True
    if haystack.find(" "+needle+" ") != -1:
        return True
    return False

In [6]:
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [ ]:
from Book1 import words 
raw_data1=raw_data[0:60000]
raw_data2=[]
cout=0
for index,raw in enumerate(raw_data1):
  word_in_raw=False
  for word in words:
    clean_raw=clean_sentence(raw)
    if smart_find(clean_raw,word['English']) and not smart_find(clean_raw,word['French']):
      print(index)
      print('\n')
      print(word)
      print('\n')
      print(clean_raw)
      print('\n')
      print('---------')
      
      word_in_raw=True
  if not word_in_raw :
    cout=cout+1
    raw_data2.append(raw)




In [ ]:
raw_data3=raw_data[0:len(raw_data)]

# for word in words:
#   z=word['English']+'\t'+word['French']
#   raw_data3.append(z)





In [ ]:
for word in words:
  z=word['English']+'\t'+word['French']
  raw_data3.append(z)

In [ ]:
# from Book1 import words 
# words2=words
# x=words1[1]['English']
# print(x.split('\/'))
for word in words1:
  word['English']=word['English'][0]
  word['French']=word['French'][0]
  word['Arabic']=word['Arabic'][0]


In [ ]:
for index, word in enumerate(words1):
   if 'Hardy-Weinberg principle'==word['English']:
       print(index)
       print(word)

In [ ]:
with open('fra_eng_sc.txt', 'w') as f:
    for raw in raw_data3:
        f.write("%s\n" % raw)

In [ ]:
print(len(words))

1445


In [11]:
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [25]:

english_sentences = [clean_sentence(pair[0]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").rstrip().lstrip() for pair in pairs]
french_sentences = [clean_sentence(pair[1]).replace("\u202f"," ").replace("\xa0"," ").replace("\u200b"," ").rstrip().lstrip() for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))


# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("french vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

Maximum length french sentence: 23
Maximum length english sentence: 17
french vocabulary is of 28264 unique words
English vocabulary is of 15182 unique words


In [ ]:
print(french_sentences[50000])

n'est-ce pas agaçant


In [26]:
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(eng_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized,max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized,max_english_len, padding = "post")




fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [27]:
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=256,)(input_sequence)

In [28]:
encoder = LSTM(128, return_sequences=False)(embedding)
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(128, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)


In [29]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 23)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 23, 256)           7235584   
                                                                 
 lstm_4 (LSTM)               (None, 128)               197120    
                                                                 
 repeat_vector_2 (RepeatVect  (None, 17, 128)          0         
 or)                                                             
                                                                 
 lstm_5 (LSTM)               (None, 17, 128)           131584    
                                                                 
 time_distributed_2 (TimeDis  (None, 17, 15182)        1958478   
 tributed)                                                 

In [ ]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(fra_pad_sentence) / BATCH_SIZE
SAVE_PERIOD = 10


from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("model/weights1.{epoch:02d}.hdf5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto', save_freq=int(SAVE_PERIOD * STEPS_PER_EPOCH))

In [ ]:
print(int(SAVE_PERIOD * STEPS_PER_EPOCH))

20000


In [30]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)


Epoch 1/10
2182/2182 [==============================] - 123s 55ms/step - loss: 2.4593 - accuracy: 0.6635
Epoch 2/10
2182/2182 [==============================] - 118s 54ms/step - loss: 2.3185 - accuracy: 0.6665
Epoch 3/10
2182/2182 [==============================] - 117s 54ms/step - loss: 2.3106 - accuracy: 0.6665
Epoch 4/10
2182/2182 [==============================] - 117s 54ms/step - loss: 2.3095 - accuracy: 0.6669
Epoch 5/10
2182/2182 [==============================] - 117s 54ms/step - loss: 2.1140 - accuracy: 0.6772
Epoch 6/10
2182/2182 [==============================] - 118s 54ms/step - loss: 1.9028 - accuracy: 0.6966
Epoch 7/10
2182/2182 [==============================] - 117s 53ms/step - loss: 1.6633 - accuracy: 0.7237
Epoch 8/10
2182/2182 [==============================] - 116s 53ms/step - loss: 1.4923 - accuracy: 0.7424
Epoch 9/10
2182/2182 [==============================] - 118s 54ms/step - loss: 1.3529 - accuracy: 0.7563
Epoch 10/10
2182/2182 [==============================] 

In [44]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)


Epoch 1/10
2182/2182 [==============================] - 122s 54ms/step - loss: 1.2000 - accuracy: 0.7738
Epoch 2/10
2182/2182 [==============================] - 118s 54ms/step - loss: 1.0655 - accuracy: 0.7874
Epoch 3/10
2182/2182 [==============================] - 117s 54ms/step - loss: 0.9909 - accuracy: 0.7956
Epoch 4/10
2182/2182 [==============================] - 117s 54ms/step - loss: 0.9313 - accuracy: 0.8030
Epoch 5/10
2182/2182 [==============================] - 117s 54ms/step - loss: 0.8821 - accuracy: 0.8094
Epoch 6/10
2182/2182 [==============================] - 118s 54ms/step - loss: 0.8385 - accuracy: 0.8155
Epoch 7/10
2182/2182 [==============================] - 118s 54ms/step - loss: 0.8019 - accuracy: 0.8210
Epoch 8/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.7691 - accuracy: 0.8257
Epoch 9/10
2182/2182 [==============================] - 117s 54ms/step - loss: 0.7399 - accuracy: 0.8301
Epoch 10/10
2182/2182 [==============================] 

In [45]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)

Epoch 1/10
2182/2182 [==============================] - 118s 54ms/step - loss: 0.6903 - accuracy: 0.8385
Epoch 2/10
2182/2182 [==============================] - 119s 54ms/step - loss: 0.6690 - accuracy: 0.8422
Epoch 3/10
2182/2182 [==============================] - 118s 54ms/step - loss: 0.6499 - accuracy: 0.8456
Epoch 4/10
2182/2182 [==============================] - 117s 53ms/step - loss: 0.6320 - accuracy: 0.8486
Epoch 5/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.6158 - accuracy: 0.8517
Epoch 6/10
2182/2182 [==============================] - 122s 56ms/step - loss: 0.6011 - accuracy: 0.8544
Epoch 7/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.5869 - accuracy: 0.8573
Epoch 8/10
2182/2182 [==============================] - 120s 55ms/step - loss: 0.5735 - accuracy: 0.8597
Epoch 9/10
2182/2182 [==============================] - 120s 55ms/step - loss: 0.5619 - accuracy: 0.8618
Epoch 10/10
2182/2182 [==============================] 

In [48]:
enc_dec_model.save('my_model_wholefile_exp.h5')



# from google.colab import files
# files.download('my_model_wholefile_exp.h5')

In [51]:
import tensorflow as tf
import keras as keras
model = tf.keras.models.load_model('my_model_wholefile_exp.h5')
model_results = model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)
 # get weights
# modelWeights = model.get_weights()
#  # get optimizer state as it was on last epoch
# modelOptimizer = model.optimizer

#  # ============ Compile Model ============
#  # redefine architecture (newModel=models.Sequential(), etc.)
# newModel=enc_dec_model = Model(input_sequence, Activation('softmax')(logits))

#  # compile
# newModel.compile(optimizer=modelOptimizer,
#                   loss=sparse_categorical_crossentropy,
#                   metrics=['accuracy'])
#  # set trained weights
# newModel.set_weights(modelWeights)

#  # ============ Resume Training ============
# model_results1 = model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)

Epoch 1/10
2134/2182 [============================>.] - ETA: 2s - loss: 0.4619 - accuracy: 0.8817

KeyboardInterrupt: ignored

In [ ]:
# import tensorflow as tf
# new_enc_dec_model = tf.keras.models.load_model('my_model.h5')



In [54]:
x=["Il faisait tellement"]
y=fra_text_tokenizer.texts_to_sequences(x)
z=pad_sequences(y, max_french_len, padding = "post")

# print(y)
z=np.reshape(z,(1,max_french_len,1))

def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

index = 8000
# print("The english sentence is: {}".format(english_sentences[index]))
print("The french sentence is: {}".format(x[0]))
print('The predicted sentence is :')
print(logits_to_sentence(enc_dec_model.predict(z)[0], eng_text_tokenizer))


The french sentence is: Il faisait tellement froid
The predicted sentence is :
it was so cold             


In [47]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=88, epochs=10)


Epoch 1/10
2182/2182 [==============================] - 120s 55ms/step - loss: 0.5392 - accuracy: 0.8664
Epoch 2/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.5296 - accuracy: 0.8682
Epoch 3/10
2182/2182 [==============================] - 121s 56ms/step - loss: 0.5199 - accuracy: 0.8702
Epoch 4/10
2182/2182 [==============================] - 121s 55ms/step - loss: 0.5111 - accuracy: 0.8719
Epoch 5/10
2182/2182 [==============================] - 119s 54ms/step - loss: 0.5028 - accuracy: 0.8734
Epoch 6/10
2182/2182 [==============================] - 120s 55ms/step - loss: 0.4947 - accuracy: 0.8751
Epoch 7/10
2182/2182 [==============================] - 120s 55ms/step - loss: 0.4867 - accuracy: 0.8768
Epoch 8/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.4799 - accuracy: 0.8781
Epoch 9/10
2182/2182 [==============================] - 119s 55ms/step - loss: 0.4729 - accuracy: 0.8797
Epoch 10/10
2182/2182 [==============================] 

In [ ]:
 # Lower case the sentence
lower_case_sent = "N'est-ce pas agaçan¡t!.".lower()
print(lower_case_sent)
# Strip punctuation
string_punctuation = "?" + "¡" + '¿'+"."+"!" ;
print(string_punctuation)

clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
print(clean_sentence)
print(string.punctuation)

n'est-ce pas agaçan¡t!.
?¡¿.!
n'est-ce pas agaçant
!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


exp

In [ ]:
#exp

input_sequence1 = Input(shape=(12,))
embedding1 = Embedding(input_dim=7188, output_dim=128,)(input_sequence1)

model1 = Model(inputs=input_sequence1 , outputs=embedding1, name="mnist_model")
model1.compile('rmsprop', 'mse')


input_data = np.array([[1,2,4,5,18,0,0,0,4,10,12,13]])

output_array = model1.predict(input_data)
print(output_array[0][0])


In [ ]:
# for layer in model1.layers: print(layer.get_config(), layer.get_weights())
#print(model1.layers[1].get_weights()[0].shape)

(7188, 128)


In [ ]:
#exp
lstm = LSTM(64, return_sequences=False)(embedding1)

lstmrepeatvector = RepeatVector(6)(lstm)
lstmdecoder = LSTM(64, return_sequences=True, dropout=0.2)(lstmrepeatvector)
lstmlogits = TimeDistributed(Dense(3733))(lstmdecoder)
modellstm = Model(inputs=input_sequence1 , outputs=lstmlogits, name="mnist_model")

In [ ]:
#exp
modellstm.compile('rmsprop', 'mse')


In [ ]:
modellstm.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modellstm, 'model')
!zip -r model.zip model

In [ ]:
#EXP
output_array = modellstm.predict(input_data)
reshape_out=np.reshape(output_array[0][0],(3733,-1))
print(reshape_out.shape)


(3733, 1)


In [ ]:
# list_of_lists = [[1,2,3],[4,5,6],[7,8,9]]

# import csv

# with open("out.csv", "w") as f:
#     wr = csv.writer(f)
   
#     wr.writerows(exppairs)

exppairs=pairs[0:10]
file = open("sample.py", "w+")

# Saving the array in a text file
for pair in exppairs :
   file.write(str(pair))
file.close()


In [ ]:
# modelexp = tf.sequential();
# modelexp.add(tf.layers.dense({ units: 1, inputShape: [1] }));

In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=3000, batch_size=12 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))

In [ ]:
modelexp.save('my_model_v21.h5')
!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(modelexp, 'model1')
!zip -r model.zip model

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import tensorflow as tf
import keras as keras
modelexp = Sequential()
# modelexp.add(Input(shape=(1,)))
modelexp.add(Dense(1,input_dim=1))

modelexp.compile(optimizer='sgd', loss=tf.keras.losses.MeanSquaredError( name="mean_squared_error"),metrics=['accuracy'])
xs = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]);
xs=np.reshape(xs,(12,1))
ys = np.array([1, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]);
ys=np.reshape(ys,(12,1))
modelexp.fit(xs, ys,epochs=300, batch_size=1 )
x1=np.array([40])
x1=np.reshape(x1,(1,1))
print(x1.shape);
print(modelexp.predict(x1))